In [ ]:
%cd ../..
%load_ext autoreload

%autoreload 2

In [ ]:
import os
from pathlib import Path
import rts
from rts.utils import dataframe_from_hdf5
import rts.features.pose as pose
import numpy as np
import pandas as pd

In [ ]:
archiva_path = "/mnt/g/ioc/data/"
os.path.isdir(archiva_path)

In [ ]:
data = dataframe_from_hdf5(archiva_path, "metadata")

In [ ]:
data.head()

In [ ]:
sequences_folder = "/mnt/g/ioc/sequences/"
extracted_sequences = os.listdir(sequences_folder)
data = data[data.guid.map(lambda x: x in extracted_sequences)]
data = data[data.duration_sec < 600]

In [ ]:
data.sport.value_counts()

In [ ]:
top_sports = data.sport.value_counts().index.tolist()
top_sports.remove("Non-Sport")

In [ ]:
data = data[data.sport.map(lambda x: x in top_sports)]

In [ ]:
N_SEQS_PER_SPORT = 100
sample = data.groupby("sport").sample(N_SEQS_PER_SPORT)
len(sample)

In [ ]:
sequences_folder = "/mnt/g/ioc/sequences/"

sequences_videos = []
for root, dirs, files in os.walk(sequences_folder):
    for file in files:
        if file.endswith('.mp4'):
            sequences_videos.append(os.path.join(root, file))

print(len(sequences_videos))

In [ ]:
data["video_path"] = data.guid.map(lambda x: [f for f in sequences_videos if x in f][0])

In [ ]:
output_folder = "data/test_poses/"

data.apply(lambda df: pose.process_video(
    pose.PifPafModel.fast, 
    Path(output_folder + "%s.jl" %df["guid"]), 
    Path(df["video_path"]), 
    skip_frame=10
), axis = 1)

In [ ]:
poses_jl = [output_folder + f for f in os.listdir(output_folder)]
poses = []
for pose_json in poses_jl:
    poses.append(pose.extract_frame_data(pose_json, 0.5))

print(len(poses))